In [1]:
%load_ext autoreload
%autoreload 2

import warnings
import pandas as pd
import numpy as np
import os
import sys # error msg, add the modules
import operator # sorting
from math import *
import matplotlib.pyplot as plt

sys.path.append('../')

warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)

import prep
from warp import *

In [2]:
trace_file = 'trace.csv'
metrics_file = 'metrics.csv'

# read trace and metrics into dataframe, from generated trace file
df_kernel_trace, df_metrics = prep.Prep_trace_metrics(trace_file, metrics_file)

In [3]:
unique_kerns = []
for index, kenname in enumerate(df_kernel_trace.Name.unique()):
    if index > 0 and 'memcpy' not in kenname:
        head = kenname.split('(')[0]
        if '<' in head:
            head = head.split('<')[0]
        #print head
    
        if 'void' in head:
            head = head.split(' ')
            #print head[1]
            head = head[1]
            
        if head not in unique_kerns:
            unique_kerns.append(head)
            print head

copy
copySharedMem
transposeNaive
transposeCoalesced
transposeNoBankConflicts
transposeCoarseGrained
transposeFineGrained
transposeDiagonal


In [4]:
target_kern_name = 'transposeNaive'
sass_result = 'transposeNaive.sm_52.sass.result'
    
out_columns = prep.init_df_columns(df_metrics)

df_kern = pd.DataFrame(columns=out_columns)

# current kernel trace
df_current = prep.GenCurKernInfo(df_kernel_trace, df_metrics,target_kern_name, out_columns)
    
# add current trace
df_kern = df_kern.append(df_current, ignore_index=True)

#df_kern.to_csv('kernel_stat.csv', index=False, encoding='utf-8')

warp_inst = WarpInst()

warp_inst.run(df_kern, sass_result)

LDG (global load) clocks (per warp) 		 = 650.0
STG (global store) clocks (per warp) 		 = 19.0
LDS (shared memory load) clocks (per warp) 	 = 0.0
STS (shared memory store) clocks (per warp) 	 = 0.0
=> Memory inst. (per warp) 			 = 669.0 (clocks)

Integer inst. (per warp) 		 = 240.0 (clocks)
FP32 inst. (per warp) 			 = 0.0 (clocks)
FP64 inst. (per warp) 			 = 0.0 (clocks)
Compute load store inst. (per warp) 	 = 30.0 (clocks)
=> Compute inst. (per warp) 		 = 270.0 (clocks)

Memory Intensive : mem / cmp = 2.47777777778


In [5]:
target_kern_name = 'transposeCoalesced'
sass_result = 'transposeCoalesced.sm_52.sass.result'
    
out_columns = prep.init_df_columns(df_metrics)

df_kern = pd.DataFrame(columns=out_columns)

# current kernel trace
df_current = prep.GenCurKernInfo(df_kernel_trace, df_metrics,target_kern_name, out_columns)
    
# add current trace
df_kern = df_kern.append(df_current, ignore_index=True)

#df_kern.to_csv('kernel_stat.csv', index=False, encoding='utf-8')

warp_inst = WarpInst()

warp_inst.run(df_kern, sass_result)

LDG (global load) clocks (per warp) 		 = 650.0
STG (global store) clocks (per warp) 		 = 19.0
LDS (shared memory load) clocks (per warp) 	 = 26.0
STS (shared memory store) clocks (per warp) 	 = 19.0
=> Memory inst. (per warp) 			 = 714.0 (clocks)

Integer inst. (per warp) 		 = 330.0 (clocks)
FP32 inst. (per warp) 			 = 0.0 (clocks)
FP64 inst. (per warp) 			 = 0.0 (clocks)
Compute load store inst. (per warp) 	 = 60.0 (clocks)
=> Compute inst. (per warp) 		 = 390.0 (clocks)

Memory Intensive : mem / cmp = 1.83076923077


In [6]:
target_kern_name = 'transposeNoBankConflicts'
sass_result = 'transposeNoBankConflicts.sm_52.sass.result'
    
out_columns = prep.init_df_columns(df_metrics)

df_kern = pd.DataFrame(columns=out_columns)

# current kernel trace
df_current = prep.GenCurKernInfo(df_kernel_trace, df_metrics,target_kern_name, out_columns)
    
# add current trace
df_kern = df_kern.append(df_current, ignore_index=True)

#df_kern.to_csv('kernel_stat.csv', index=False, encoding='utf-8')

warp_inst = WarpInst()

warp_inst.run(df_kern, sass_result)

LDG (global load) clocks (per warp) 		 = 650.0
STG (global store) clocks (per warp) 		 = 19.0
LDS (shared memory load) clocks (per warp) 	 = 26.0
STS (shared memory store) clocks (per warp) 	 = 19.0
=> Memory inst. (per warp) 			 = 714.0 (clocks)

Integer inst. (per warp) 		 = 360.0 (clocks)
FP32 inst. (per warp) 			 = 0.0 (clocks)
FP64 inst. (per warp) 			 = 0.0 (clocks)
Compute load store inst. (per warp) 	 = 60.0 (clocks)
=> Compute inst. (per warp) 		 = 420.0 (clocks)

Memory Intensive : mem / cmp = 1.7


In [7]:
target_kern_name = 'transposeCoarseGrained'
sass_result = 'transposeCoarseGrained.sm_52.sass.result'
    
out_columns = prep.init_df_columns(df_metrics)

df_kern = pd.DataFrame(columns=out_columns)

# current kernel trace
df_current = prep.GenCurKernInfo(df_kernel_trace, df_metrics,target_kern_name, out_columns)
    
# add current trace
df_kern = df_kern.append(df_current, ignore_index=True)

#df_kern.to_csv('kernel_stat.csv', index=False, encoding='utf-8')

warp_inst = WarpInst()

warp_inst.run(df_kern, sass_result)

LDG (global load) clocks (per warp) 		 = 650.0
STG (global store) clocks (per warp) 		 = 19.0
LDS (shared memory load) clocks (per warp) 	 = 0.0
STS (shared memory store) clocks (per warp) 	 = 19.0
=> Memory inst. (per warp) 			 = 688.0 (clocks)

Integer inst. (per warp) 		 = 315.0 (clocks)
FP32 inst. (per warp) 			 = 0.0 (clocks)
FP64 inst. (per warp) 			 = 0.0 (clocks)
Compute load store inst. (per warp) 	 = 45.0 (clocks)
=> Compute inst. (per warp) 		 = 360.0 (clocks)

Memory Intensive : mem / cmp = 1.91111111111


In [8]:
target_kern_name = 'transposeFineGrained'
sass_result = 'transposeFineGrained.sm_52.sass.result'
    
out_columns = prep.init_df_columns(df_metrics)

df_kern = pd.DataFrame(columns=out_columns)

# current kernel trace
df_current = prep.GenCurKernInfo(df_kernel_trace, df_metrics,target_kern_name, out_columns)
    
# add current trace
df_kern = df_kern.append(df_current, ignore_index=True)

#df_kern.to_csv('kernel_stat.csv', index=False, encoding='utf-8')

warp_inst = WarpInst()

warp_inst.run(df_kern, sass_result)

LDG (global load) clocks (per warp) 		 = 650.0
STG (global store) clocks (per warp) 		 = 19.0
LDS (shared memory load) clocks (per warp) 	 = 26.0
STS (shared memory store) clocks (per warp) 	 = 19.0
=> Memory inst. (per warp) 			 = 714.0 (clocks)

Integer inst. (per warp) 		 = 270.0 (clocks)
FP32 inst. (per warp) 			 = 0.0 (clocks)
FP64 inst. (per warp) 			 = 0.0 (clocks)
Compute load store inst. (per warp) 	 = 60.0 (clocks)
=> Compute inst. (per warp) 		 = 330.0 (clocks)

Memory Intensive : mem / cmp = 2.16363636364


In [9]:
target_kern_name = 'transposeDiagonal'
sass_result = 'transposeDiagonal.sm_52.sass.result'
    
out_columns = prep.init_df_columns(df_metrics)

df_kern = pd.DataFrame(columns=out_columns)

# current kernel trace
df_current = prep.GenCurKernInfo(df_kernel_trace, df_metrics,target_kern_name, out_columns)
    
# add current trace
df_kern = df_kern.append(df_current, ignore_index=True)

#df_kern.to_csv('kernel_stat.csv', index=False, encoding='utf-8')

warp_inst = WarpInst()

warp_inst.run(df_kern, sass_result)

LDG (global load) clocks (per warp) 		 = 650.0
STG (global store) clocks (per warp) 		 = 19.0
LDS (shared memory load) clocks (per warp) 	 = 26.0
STS (shared memory store) clocks (per warp) 	 = 19.0
=> Memory inst. (per warp) 			 = 714.0 (clocks)

Integer inst. (per warp) 		 = 735.0 (clocks)
FP32 inst. (per warp) 			 = 15.0 (clocks)
FP64 inst. (per warp) 			 = 0.0 (clocks)
Compute load store inst. (per warp) 	 = 60.0 (clocks)
=> Compute inst. (per warp) 		 = 810.0 (clocks)

Compute Intensive : cmp / mem = 1.13445378151
